In [ ]:
import cPickle
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from IPython import display
import scipy.sparse as sp
from scipy.stats import pearsonr
import os
os.chdir('/Users/zbutler/research/fire_prediction')
from geometry.get_xys import append_xy
from prediction.fire_clustering import cluster_fires
from geometry.grid_conversion import ak_bb
from plotting.plot_map import draw_map_nogrid_static
import geopandas as gpd
%matplotlib inline

In [ ]:
# Load lightning
light = gpd.read_file('data/lightning/TOALightning2012Thru2015.shp')
print "loaded dataset with %d rows" % len(light)
light.iloc[0:10]

In [ ]:
# See how many fires are in ak_bb
in_bb = []
for row in light.index:
    lat = light.loc[row].LATITUDE
    lon = light.loc[row].LONGITUDE
    if (ak_bb[0] < lat < ak_bb[1]) and (ak_bb[2] < lon < ak_bb[3]):
        in_bb.append(row)
print "%d in Alaska" % len(in_bb)

In [ ]:
ak_light = light.loc[in_bb]
print "created %d len db" % len(ak_light)
ak_light.STRIKETIME.unique()

In [ ]:
# Create a nice pandas database with the date split up into separate fields
pd_dict = dict()
pd_dict['lat'] = []
pd_dict['lon'] = []
pd_dict['day'] = []
pd_dict['month'] = []
pd_dict['year'] = []
pd_dict['amplitude'] = []
pd_dict['millisec'] = []
pd_dict['stroketype'] = []
for row in ak_light.index:
    pd_dict['lat'].append(ak_light.loc[row].LATITUDE)
    pd_dict['lon'].append(ak_light.loc[row].LONGITUDE)
    year,month,day = map(int, ak_light.loc[row].LOCALDATET.split('-'))
    pd_dict['day'].append(day)
    pd_dict['month'].append(month)
    pd_dict['year'].append(year)
    pd_dict['amplitude'].append(ak_light.loc[row].AMPLITUDE)
    pd_dict['millisec'].append(ak_light.loc[row].MILLISECON)
    pd_dict['stroketype'].append(ak_light.loc[row].STROKETYPE)
ak_l = pd.DataFrame(pd_dict)
ak_l.iloc[0:10]

In [ ]:
# Summary graphs for lightning, also save them to disk for drive purposes!
cloud_l = ak_l[ak_l.stroketype==u'CLOUD_STROKE']
ground_l = ak_l[ak_l.stroketype==u'GROUND_STROKE']
month = 3
year = 2012
n_light_arr = []
n_light_cloud = []
n_light_ground = []
yearmonth_arr = []
while year < 2015 or month < 10:
    n_light = len(ak_l[(ak_l.year == year) & (ak_l.month == month)])
    n_light_arr.append(n_light)
    n_light_cloud.append(len(cloud_l[(cloud_l.year == year) & (cloud_l.month == month)]))
    n_light_ground.append(len(ground_l[(ground_l.year == year) & (ground_l.month == month)]))
    yearmonth_arr.append(year + (month/12.))
    month += 1
    if month > 12:
        month = 1
        year += 1

plt.plot(yearmonth_arr, n_light_arr)
plt.xlabel('Time')
plt.ylabel('Number of lightning events in Alaska')
plt.savefig('pics/lightning_per_month.png')
plt.show()
plt.close()

plt.hist(ak_l.amplitude, bins=50)
plt.xlabel('Amplitude')
plt.ylabel('Frequency of bin')
plt.savefig('pics/lightning_amplitude_hist.png')
plt.show()
plt.close()

plt.hist(ak_l.millisec, bins=50)
plt.xlabel('Milliseconds of lightning')
plt.ylabel('Frequency of bin')
plt.savefig('pics/lightning_millisec_hist.png')
plt.show()
plt.close()

# Do all these plots by stroke type
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
f.tight_layout()
ax1.plot(yearmonth_arr, n_light_cloud)
ax1.set_xlabel('Time')
ax1.set_ylabel('Number of cloud lightning events in Alaska')
ax2.plot(yearmonth_arr, n_light_ground)
ax2.set_xlabel('Time')
ax2.set_ylabel('Number of ground lightning events in Alaska')
plt.savefig('pics/lightning_per_month_by_type.png')
plt.show()
plt.close()

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
f.tight_layout()
ax1.hist(cloud_l.amplitude, bins=50)
ax1.set_xlabel('Amplitude')
ax1.set_ylabel('Frequency of bin (cloud)')
ax2.hist(ground_l.amplitude, bins=50)
ax2.set_xlabel('Amplitude')
ax2.set_ylabel('Frequency of bin (ground)')
plt.savefig('pics/lightning_amplitude_hist_by_type.png')
plt.show()
plt.close()

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
f.tight_layout()
ax1.hist(cloud_l.millisec, bins=50)
ax1.set_xlabel('Milliseconds of lightning')
ax1.set_ylabel('Frequency of bin (cloud)')
ax2.hist(ground_l.millisec, bins=50)
ax2.set_xlabel('Milliseconds of lightning')
ax2.set_ylabel('Frequency of bin (ground)')
plt.savefig('pics/lightning_millisec_hist_by_type.png')
plt.show()
plt.close()

In [ ]:
print ak_l.stroketype.unique()
plt.figure(figsize=(7,14))
mp = Basemap(projection="merc",
              lat_0=ak_bb[0], lon_0=ak_bb[2],
              llcrnrlat=ak_bb[0],
              llcrnrlon=ak_bb[2],
              urcrnrlat=ak_bb[1],
              urcrnrlon=ak_bb[3],
              resolution='i')

year = 2012
month = 3
prev_lats = []
prev_longs = []
while year < 2015 or month < 10:
    month_light = ak_l[(ak_l.month == month) & (ak_l.year == year)]
    display.clear_output(wait=True)
    plt.figure(figsize=(7,14))
    mp.drawmapboundary(fill_color='aqua')
    mp.fillcontinents(color='yellow',lake_color='aqua')
    mp.drawcoastlines()
    if len(prev_lats):
        mp.plot(prev_longs, prev_lats, 'ko')
    if len(month_light):
        today_longs, today_lats = mp(np.array(month_light.lon), np.array(month_light.lat))
        mp.plot(today_longs, today_lats, 'ro')
        prev_lats += list(today_lats)
        prev_longs += list(today_longs)
    plt.title("Lightning in Alaska on %d, %d" %(month, year))
    plt.show()
    time.sleep(.8)
    month += 1
    if month > 12:
        month = 1
        year += 1

In [ ]:
with open('data/ak_fires.pkl') as fpkl:
    ak_fires = cPickle.load(fpkl)

In [ ]:
from util.daymonth import increment_day
day = 1
month = 1
year = 2013

light = []
fire = []
while year == 2013:
    light.append(len(ground_l[(ground_l.day==day) & (ground_l.month==month) & (ground_l.year==year)]))
    fire.append(len(ak_fires[(ak_fires.day==day) & (ak_fires.month==month) & (ak_fires.year==year)]))
    year, month, day = increment_day(year, month, day)
    
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10,8))
ax1 = plt.subplot(211)
plt.plot(fire, 'r--')
plt.title('Number of detections')

ax2 = plt.subplot(212, sharex=ax1)
plt.plot(light, 'b--')
plt.title('Number of ground lightning strikes')
fig.tight_layout()
plt.show()

In [ ]:
# Save ground_l
with open("data/lightning/ground_lightning.pkl", 'w') as fpkl:
    cPickle.dump(ground_l, fpkl)